TSG112 - Active Directory Pre-Deployment Checks
===============================================

Description
-----------

This notebook will validate a Big Data Cluster (BDC) configuration is
valid for an Active Directory (AD) deployment. It will check the
following:

-   Configuration settings in `bdc.json` and `control.json` are valid
-   DNS is properly configured
-   The AD organizational unit (OU) the BDC is being deployed into is
    setup correctly
-   Permissions for the deployment account are correct

### Linux Prerequisites

-   `Kinit` as the BDC admin account

<!-- -->

        kinit <user>

-   Install `adutil` on Ubuntu 20.04

<!-- -->

        wget -qO- https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
        sudo add-apt-repository "(wget -qO- https://packages.microsoft.com/config/ubuntu/20.04/mssql-server-preview.list)"
        sudo apt-get update
        sudo apt-get install -y adutil

### Windows Prerequisites

-   Install the Active Directory (AD) module for Powershell by following
    the instructions
    [here](https://docs.microsoft.com/en-us/powershell/module/addsadministration/?view=win10-ps).

### Parameters

The parameters this notebook needs can be set here, or the notebook will
prompt for input when they are needed.

In [ ]:
configDirPath = ""
adDeploymentUsername = ""
adDeploymentPassword = ""
notebookCanCreateAccountInOU = ""

Read Configs
------------

Start by inputting the path to the directory holding the BDC config
files (`bdc.json` and `control.json`) as well as the username of the AD
service account that the BDC will deploy as:

In [ ]:
# Setup global variables, imports, and functions
import getpass
import json
import os
import platform
import shutil
import subprocess

ouDistinguishedName = ''
dnsIpAddresses = ''
domainControllerFullyQualifiedDns = ''
domainDnsName = ''
clusterAdmins = ''
clusterUsers = ''
appOwners = ''
subdomain = ''
accountPrefix = ''

# Run a subprocess command and return outputs
def runSubprocessCommand(executable, cmd):
  process = subprocess.Popen(cmd, shell=True, executable=executable,
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()

  return stdout.strip().decode("UTF-8"), stderr.strip().decode("UTF-8")


# Run a powershell command and return outputs
def runPowershellCommand(cmd):
  return runSubprocessCommand(shutil.which('powershell'), cmd)


# Run a bash command and return outputs
def runBashCommand(cmd):
  return runSubprocessCommand('/bin/bash', cmd)

In [ ]:
# Begin config test
configTestsPassed = False
loadedBdcJson = False
loadedControlJson = False

# Get the config directory from the user
print("Config directory: ")

if configDirPath == "":
  configDirPath = input().strip('"')
else:
  print("")

print(configDirPath + "\n")

# Pull AD config values from control.json
try:
  with open(os.path.join(configDirPath, "control.json")) as controlJsonFile:
    controlConfig = json.load(controlJsonFile)

    if controlConfig['security'] \
        and controlConfig['security']['activeDirectory']:
      adData = controlConfig['security']['activeDirectory']
      ouDistinguishedName = adData['ouDistinguishedName']
      dnsIpAddresses = adData['dnsIpAddresses']
      domainControllerFullyQualifiedDns = \
        adData['domainControllerFullyQualifiedDns']
      domainDnsName = adData['domainDnsName']
      clusterAdmins = adData['clusterAdmins']
      clusterUsers = adData['clusterUsers']

      if 'subdomain' in adData:
        subdomain = adData['subdomain']
      else:
        subdomain = 'test'

      if 'accountPrefix' in adData:
        accountPrefix = adData['accountPrefix']
      else:
        accountPrefix = subdomain

      try:
        appOwners = adData['appOwners']
      except KeyError:
        appOwners = []

      if ouDistinguishedName and dnsIpAddresses \
          and domainControllerFullyQualifiedDns and domainDnsName \
          and clusterAdmins and clusterUsers:
        loadedControlJson = True
except IOError:
  print("Error: could not read {0}/control.json".format(configDirPath))
except KeyError as err:
  print("Error: missing settings in control.json: {0}. Ensure the following section is in control.json:".format(err) + 
'''
 "security": {
      "activeDirectory": {
          "ouDistinguishedName": "OU=bdc,DC=contoso,DC=com",
          "dnsIpAddresses": ["10.10.10.10"],
          "domainControllerFullyQualifiedDns": ["kdc.contoso.com"],
          "domainDnsName": "contoso.com",
          "clusterAdmins": ["bdcAdminGroup"],
          "clusterUsers": ["bdcUsersGroup"]
      }
  }
''')

masterDnsFqdn = ''

# Pull AD config values from bdc.json
try:
  with open(os.path.join(configDirPath, "bdc.json")) as bdcJsonFile:
    bdcJsonFile = json.load(bdcJsonFile)
    masterDnsFqdn = bdcJsonFile['spec']['resources']['master']\
                               ['spec']['endpoints'][0]['dnsName']

    if masterDnsFqdn:
      loadedBdcJson = True
except IOError:
  print("Error: could not read {0}/bdc.json".format(configDirPath))
except KeyError:
  print('Error: missing settings in bdc.json. The master resource needs the '
        'property dnsName to be set')
except IndexError:
  print('Error: missing settings in bdc.json. Master resource needs an '
        'endpoint value')

# Get BDC deployment account info
if adDeploymentUsername == "" and 'DOMAIN_SERVICE_ACCOUNT_USERNAME' in os.environ:
  adDeploymentUsername = os.environ['DOMAIN_SERVICE_ACCOUNT_USERNAME']

if adDeploymentPassword == "" and 'DOMAIN_SERVICE_ACCOUNT_PASSWORD' in os.environ:
  adDeploymentPassword = os.environ['DOMAIN_SERVICE_ACCOUNT_PASSWORD']

print("BDC AD Domain Service Account Username: ")

if adDeploymentUsername == "":
  adDeploymentUsername = input().strip('"')
else:
  print("")

print(adDeploymentUsername + "\n")

print("BDC AD Domain Service Account Password: ")

if adDeploymentPassword == "":
  adDeploymentPassword = getpass.getpass()
else:
  print("")

print("********\n")

canCreateAccountInOU = notebookCanCreateAccountInOU.lower() in ['true', 'yes']

# On Linux, ask if user is ok with us creating a user and group to validate OU
# permissions
if ouDistinguishedName:
  if notebookCanCreateAccountInOU != "":
    print(f"Notebook can create accounts in OU ({ouDistinguishedName}):\n")
    print(f"{canCreateAccountInOU}\n")
  elif platform.system().lower() != 'windows':
    answer = ""

    while answer not in ['yes', 'no']:
      print('To test permissions of the BDC AD domain service account, the '
            'notebook needs to create the user adNotebookUser and the group '
            'adNotebookGroup in the OU specified in control.json ({0}). This '
            'will delete these accounts even if they are outside the OU. '
            'Disabling this will test all other aspects, but won\'t give full '
            'coverage. Do you agree to the user and group being created? '
            '[yes/no]'.format(ouDistinguishedName))

      rawAnswer = input()
      print(rawAnswer)
      answer = rawAnswer.strip().strip('"').lower()
      canCreateAccountInOU = answer == 'yes'

    if canCreateAccountInOU:
      print('\nTesting permissions of the BDC AD domain service account will be '
            'executed\n')
    else:
      print('\nTesting permissions of the BDC AD domain service account will be '
            'skipped\n')

# Check loaded config values
if not loadedBdcJson or not loadedControlJson:
  if not ouDistinguishedName:
    print('security.activeDirectory.ouDistinguishedName not specified in '
          'control.json')
  elif not dnsIpAddresses:
    print('security.activeDirectory.dnsIpAddresses not specified in '
          'control.json')
  elif not domainControllerFullyQualifiedDns:
    print('security.activeDirectory.domainControllerFullyQualifiedDns not '
          'specified in control.json')
  elif not domainDnsName:
    print('security.activeDirectory.domainDnsName not specified in '
          'control.json')
  elif not clusterAdmins:
    print('security.activeDirectory.clusterAdmins not specified in '
          'control.json')
  elif not clusterUsers:
    print('security.activeDirectory.clusterUsers not specified in '
          'control.json')
  elif not masterDnsFqdn:
    print('spec.resources.master.spec.endpoints[0].dnsName not specified in '
          'bdc.json')
  else:
    print('Failed to load config files. Please check directory provided has '
          'both bdc.json and control.json.')
elif not adDeploymentUsername or adDeploymentUsername == '':
  print("Error: Must specify BDC AD domain service account's username")
elif not adDeploymentPassword or adDeploymentPassword == '':
  print("Error: Must specify BDC AD domain service account's password")
else:
  configTestsPassed = True

  # On Linux systems, check that an AD user has logged in
  if platform.system().lower() != 'windows':
    klistOut, klistErr = runBashCommand('klist')

    if not klistOut or klistOut == '' or (klistErr and klistErr != ''):
      print('Error: You must kinit as an AD user before running this notebook')
      configTestsPassed = False

if configTestsPassed:
  print('Success')

FIRST_CELL_RUN = True

DNS Checks
----------

Next, let’s check the DNS settings. This notebook will validate the
following:

-   The IP and FQDN of the domain controller in control.json match
-   Reverse DNS entries exist for the domain controllers
-   Both the short name and FQDN of the domain resolve with DNS

In [ ]:
# Prepare for DNS checks
import socket

# Print a warning or error
def printMessage(message, isError):
  if isError:
    print(f"Error: {message}")
  else:
    print(f"Warning: {message}")

# Check a hostname resolves to an IP and then back to the same hostname. If it
# is a domain name (short or fqdn), we ensure the returned name is from the
# correct domain
def dnsChecks(hostname, isDomainName, errorIsFatal=True):
  res = False
  ipAddr = '0.0.0.0'

  try:
    ipAddr = socket.gethostbyname(hostname)
  except socket.gaierror as err:
    printMessage(f"Failed to lookup DNS entry for '{hostname}'. Ensure there is a host record for '{hostname}': {err}", errorIsFatal)
    return False

  if ipAddr != '0.0.0.0':
    reverseLookupHost = None

    try:
      reverseLookupHost = socket.gethostbyaddr(ipAddr)[0]
    except socket.gaierror as err:
      printMessage(f"Failed to perform reverse DNS lookup for '{ipAddr}'. Ensure the DNS server has a PTR record for '{ipAddr}': {err}", errorIsFatal)
      return False

    if reverseLookupHost:
      reverseLookupDotIndex = reverseLookupHost.find('.')
      reverseLookupDotIndex2 = \
        reverseLookupHost.find('.', reverseLookupDotIndex + 1)

      if reverseLookupDotIndex != -1:
        dotIndex = hostname.find('.')

        if dotIndex == -1:
          # If short name (no '.'), then it should be first or second value
          if isDomainName:
            shortReverseLookup = reverseLookupHost[reverseLookupDotIndex + 1: 
                                                   reverseLookupDotIndex2]
            res = hostname.lower() == shortReverseLookup.lower()

            if not res:
              printMessage("Reverse lookup of {0} returned a result from {1} domain "
                    "when {2} domain was expected. Check that the host records"
                    "in DNS are correct and point to the correct domain"\
                      .format(ipAddr, shortReverseLookup, hostname), errorIsFatal)
          else:
            shortReverseLookup = reverseLookupHost[0:reverseLookupDotIndex]
            res = hostname.lower() == shortReverseLookup.lower()

            if not res:
              printMessage("Reverse lookup of {0} returned {1} when {2} was "
                    "expected. Ensure the PTR records on the DNS server are "
                    "correct".format(ipAddr, shortReverseLookup, hostname), errorIsFatal)
        else:
          # Fully qualified domain is passed. Should be full name or subset (if
          # a domain)
          if isDomainName:
            reverseDomainName = reverseLookupHost[reverseLookupDotIndex + 1:]
            res = hostname.lower() == reverseDomainName.lower()

            if not res:
              printMessage("Reverse lookup of {0} returned a result from {1} domain "
                    "when {2} domain was expected. Ensure the PTR records on "
                    "the DNS server are correct and point to the correct "
                    "domain".format(ipAddr, reverseDomainName, hostname), errorIsFatal)
          else:
            res = hostname.lower() == reverseLookupHost.lower()

            if not res:
              printMessage("Reverse lookup of {0} returned {1} when {2} was "
                    "expected. Ensure the PTR records on the DNS server are"
                    "correct".format(ipAddr, reverseLookupHost, hostname), errorIsFatal)
      else:
        # Have a short name from reverse lookup
        printMessage("Reverse lookup of {0} returned {1} which should have been an "
              "FQDN. Check the PTR records on the DNS server and ensure they"
              "resolve to an FQDN".format(ipAddr, reverseLookupHost), errorIsFatal)

  return res


def runDnsTest():
  res = False

  if configTestsPassed:
    # Check the domain controller FQDN resolves
    try:
      ipAddr = socket.gethostbyname(domainControllerFullyQualifiedDns[0])
    except socket.gaierror as err:
      printMessage(f"Failed to lookup DNS entry for '{domainControllerFullyQualifiedDns[0]}'. Ensure there is a host record for '{domainControllerFullyQualifiedDns[0]}': {err}", True)
      return False

    if ipAddr and ipAddr != '' and ipAddr != '0.0.0.0':
      # Check RDNS entries for the domain controller
      hostname = socket.gethostbyaddr(ipAddr)[0]

      if hostname.lower() == domainControllerFullyQualifiedDns[0].lower():
        dotIndex = domainDnsName.find('.')
        shortDomainName = domainDnsName[:dotIndex]

        # Check DNS entries for the short domain name and FQDN of the domain
        dnsChecks(shortDomainName, True, False)

        if dnsChecks(domainDnsName, True):
          res = True
      else:
        print("Reverse DNS entries for {0} are incorrect. Should be {1} but "
              "is {2}. Please fix the PTR records on the DNS server"\
              .format(ipAddr, domainControllerFullyQualifiedDns[0], hostname))
    else:
      print("An invalid IP ('{0}') was found for the domain controller FQDN "\
            "('{1}'). Make sure that the correct DNS records exist for the "\
            "FQDN."\
            .format(ipAddr, domainControllerFullyQualifiedDns[0]))
  else:
    print("Cannot run tests until config has been loaded")

  return res

In [ ]:
# Run DNS tests
shouldRunTests = False
dnsTestsSucceeded = False

try:
  FIRST_CELL_RUN
  shouldRunTests = True
except NameError:
  print("Error: First code cell must be run before this cell")

if shouldRunTests:
  dnsTestsSucceeded = runDnsTest()

if dnsTestsSucceeded:
  print("Success")

Existing Objects Checks
-----------------------

Next, let’s check the OU specified in control.json. This will validate:

-   The OU is valid
-   Accounts that the BDC will create in the OU don’t already exist

In [ ]:
# Prepare for existing object checks
from enum import Enum
import os
import platform
import shutil
import subprocess


# Check if adutil is installed
def adutilInstalled():
  (pipeRead, pipeWrite) = os.pipe()
  stdout, _ = runBashCommand("whereis adutil")
  words = stdout.split(':')

  return len(words) == 2 and words[1] != ''


# Check if an account exists
def accountExistsLinux(accountName):
  exists = False
  error = "Function didn't finish"

  # Call adutil and capture output which will be "true", "false", or an error
  (pipeRead, pipeWrite) = os.pipe()
  stdout, stderr = runBashCommand("adutil account exists {0}"\
                                  .format(accountName))
  resLines = stdout.split('\n')
  strRes = resLines[0].lower()

  # Parse output
  if strRes == 'true':
    exists = True
    error = ''
  elif strRes == 'false':
    exists = False
    error = ''
  else:
    error = stdout

  return exists, error


# Check if an OU exists on Linux
def ouExistsLinux(ouName):
  stdout, _ = runBashCommand("adutil ou exists --distname {0}".format(ouName))

  if stdout.strip().lower() == 'true':
    return True, ''
  elif stdout.strip().lower() == 'false':
    return False, ''
  else:
    return False, stdout


WELL_KNOWN_GROUPS = [
  "dmsvc",
  "desvc"
]

WELL_KNOWN_USERS = [
  "apst",
  "ctrl",
  "dmc*-*",
  "dmmp-*",
  "dec*-*",
  "demp-*",
  "hdt*-*",
  "hdnn-*",
  "hvsh-*",
  "htgw",
  "htnn",
  "htsh",
  "hts*",
  "htt*",
  "htzk",
  "jnzk-*",
  "kmnn-*",
  "knox-*",
  "ldap",
  "lvsh-*",
  "sqc*",
  "sqc*-*",
  "sqd-*",
  "sqmp",
  "sqmp-*",
  "sqs-*",
  "ngxm",
  "shsh-*",
  "yns*-*",
  "ynt*-*",
  "yrsh-*"
]


class AccountType(Enum):
  User = 1
  Group = 2
  OU = 3
  Unknown = 4


# Check if an AD object exists from Windows client
def objectExistsWindows(accountName, accountType):
  exists = True
  error = "Function didn't finish"

  stdout = ""
  stderr = ""

  # Lookup object based on type
  if accountType == AccountType.Group:
    stdout, stderr = runPowershellCommand(
      "get-adgroup -Filter 'samAccountName -Like \\\"{0}\\\"'".format(
        accountName))
  elif accountType == AccountType.User:
    stdout, stderr = runPowershellCommand(
      "get-aduser -Filter 'samAccountName -Like \\\"{0}\\\"'".format(
        accountName))
  elif accountType == AccountType.OU:
    stdout, stderr = runPowershellCommand(
      "get-adorganizationalunit -Filter 'distinguishedname -Like \\\"{0}\\\"'"\
        .format(accountName))
  else:
    return exists, 'Unknown account type'

  # Parse output
  if stdout != '':
    # If anything is printed to stdout, then an object has been found
    exists = True
    error = ''
  elif stderr != '':
    # If there is an error, return it
    error = stderr
  else:
    # No object found and no errors means the object doesn't exist
    exists = False
    error = ''

  return exists, error


# Check that no account in a list exists
def checkListAccountsDontExistWindows(accountList, accountType):
  noAccountsExist = True
    
  for staticAccountName in accountList:
    accountName = accountPrefix + '-' + staticAccountName
    exists, error = objectExistsWindows(accountName, accountType)

    if exists or error != '':
      noAccountsExist = False

      if error != '':
        print("Error looking up account {0}: {1}".format(accountName, error))
      else:
        print("Account {0} exists and it should not. Please delete this "
              "account then run the notebook cell again".format(accountName))
      
      break

  return noAccountsExist

In [ ]:
# Linux tests for existing AD objects
def linuxExistingObjectsTest():
  noAccountsExist = False
  ouExists = False

  if adutilInstalled():
    noAccountsExist = True

    # Check no well-known groups currently exist
    wellKnownAccounts = WELL_KNOWN_GROUPS + WELL_KNOWN_USERS

    for staticAccountName in wellKnownAccounts:
      account = accountPrefix + '-' + staticAccountName
      exists, error = accountExistsLinux(account)

      if exists or error != '':
        noAccountsExist = False

        if error != '':
          print("Error looking up account {0}: {1}. Ensure you have run kinit, "
                "have permissions to query AD, and can contact the domain "
                "controller".format(account, error))
        else:
          print("Account {0} exists and it should not. Remove it then run the "
                "code cell again".format(account))

        break

    ouExists, error = ouExistsLinux(ouDistinguishedName)

    if error and error != '':
      print("Failed to check if OU '{0}' exists: {1}. Ensure you have run "
            "kinit, have permissions to query AD, and can contact the domain "
            "controller".format(ouDistinguishedName, error))
    elif not ouExists:
      print("OU '{0}' doesn't exist, but it should. Please create it then run "
            "the code cell again".format(ouDistinguishedName))
  else:
    print("Adutil is not installed but is required. Please install it then run"
          "the code cell again")

  return noAccountsExist and ouExists


# Windows tests for existing AD objects
def windowsExistingObjectsTest():
  res = False

  # Check none of the well-known accounts exist
  noAccountsExist = checkListAccountsDontExistWindows(WELL_KNOWN_GROUPS,
                                                      AccountType.Group)
  noAccountsExist = noAccountsExist\
    and checkListAccountsDontExistWindows(WELL_KNOWN_USERS, AccountType.User)

  # Check the OU exists
  ouExists, error = objectExistsWindows(ouDistinguishedName, AccountType.OU)

  if not error:
    res = noAccountsExist and ouExists

    if not ouExists:
      print("The organizational unit specified in control.json ({0}) does not "
            "exist, but it should. Please create it then run the code cell "
            "again".format(ouDistinguishedName))
  else:
    print("Error looking up OU {0}: {1}".format(ouDistinguishedName, error))

  return res


# Test for existing AD objects
existingObjectsChecksSucceeded = False
shouldRunTests = False

try:
  FIRST_CELL_RUN
  shouldRunTests = True
except NameError:
  print("Error: First code cell must be run before this cell")

if shouldRunTests:
  if platform.system().lower() == 'windows':
    existingObjectsChecksSucceeded = windowsExistingObjectsTest()
  else:
    existingObjectsChecksSucceeded = linuxExistingObjectsTest()


if existingObjectsChecksSucceeded:
  print ("Success")

Account Checks
--------------

The next step is to check that all AD users and groups used during
deployment are valid. The code block below will check the following:

-   No groups supplied in control.json are domain-local groups
-   All accounts specified in control.json are enabled
-   The username and password of the AD BDC deployment account are valid
-   The AD BDC deployment account has the correct perms on the OU the
    BDC will deploy into

In [ ]:
# Prepare for checking deployment accounts
from enum import Enum
from IPython.display import display, Markdown
import platform
import shutil
import subprocess
import uuid

# Get the type of an account from a windows client
def getAccountTypeWindows(accountName):
  type = AccountType.Unknown
  error = 'Function did not finish'

  # Call powershell to get group type
  stdout, stderr = runPowershellCommand(
    "(get-adobject -Filter '(samAccountName -Like \\\"{0}\\\")').ObjectClass"\
      .format(accountName))

  # Parse output
  if stdout != '':
    if stdout.lower() == 'user':
      type = AccountType.User
      error = ''
    elif stdout.lower() == 'group':
      type = AccountType.Group
      error = ''
    else:
      type = AccountType.Unknown
      error = "Unknown type. Found type '{0}'".format(stdout.lower())
  else:
    # If there is an error, return it
    error = stderr

  return type, error


# Get the type of an account from a linux client
def getAccountTypeLinux(accountName):
  type = AccountType.Unknown
  error = ''

  stdout, _ = runBashCommand("adutil account gettype {0}".format(accountName))
  sanitizedStdout = stdout.strip().lower()

  if sanitizedStdout == 'user':
    type = AccountType.User
  elif sanitizedStdout == 'group':
    type = AccountType.Group
  else:
    type = AccountType.Unknown
    error = stdout

  return type, error


# Get the type of an account
def getAccountType(accountName):
  if platform.system().lower() == 'windows':
    return getAccountTypeWindows(accountName)
  else:
    return getAccountTypeLinux(accountName)


# Check that a group is not domain local from a Windows client
def checkGroupNotLocalWindows(groupName):
  isLocal = True
  error = "Function didn't finish"

  # Lookup group type
  stdout, stderr = runPowershellCommand("(get-adgroup '{0}').GroupScope"\
    .format(groupName))

  # Parse output
  if stdout != '':
    isLocal = stdout.lower() == 'domainlocal'
    error = ''
  else:
    # If there is an error, return its
    error = stderr

  return not isLocal, error


# Check that a group is not domain local from a Linux client
def checkGroupNotLocalLinux(group):
  stdout, _ = runBashCommand("adutil group gettype {0}".format(group))
  sanitizedStdout = stdout.strip().lower()

  if sanitizedStdout == 'domainlocal':
    return False, ''
  elif sanitizedStdout == 'global' or sanitizedStdout == 'universal':
    return True, ''
  else:
    return False, stdout


# Check that a group is not domain local
def checkGroupNotLocal(group):
  if platform.system().lower() == 'windows':
    return checkGroupNotLocalWindows(group)
  else:
    return checkGroupNotLocalLinux(group)


# Check that no group in a list is a DomainLocal group
def checkNoLocalGroupsInList(groupList):
  noGroupsLocal = True

  for group in groupList:
    groupType, error = getAccountType(group)

    if error != '':
      noGroupsLocal = False
      print("Failed to find type of account '{0}': {1}".format(group, error))
    elif groupType == AccountType.Group:
      notLocal, error = checkGroupNotLocal(group)

      if error and error != '':
        noGroupsLocal = False
        print("Failed to find if group '{0}' was local or not: {1}"\
              .format(group, error))
        break
      elif not notLocal:
        noGroupsLocal = False
        print("Group '{0}' specified in control.json is a DomainLocal group "
              "which is not supported. Please either change the type of group "
              "or specify a different AD identity in control.json"\
              .format(group))
        break

  return noGroupsLocal

In [ ]:
# Wrapper on the powershell commands to check permissions on an OU
def powershellAccountHasPermsOnOUWindows(username, ou, inheritance,
                                        objects, rights):
  cmd = """Import-Module ActiveDirectory

$OBJECT_TYPE_MAP = @{{
    \\"bf967a9c-0de6-11d0-a285-00aa003049e2\\" = \\"group\\";
    \\"bf967aba-0de6-11d0-a285-00aa003049e2\\" = \\"user\\";
    \\"bf967a86-0de6-11d0-a285-00aa003049e2\\" = \\"computer\\";
    \\"00000000-0000-0000-0000-000000000000\\" = \\"all\\";
}}

Add-Type -TypeDefinition @\\"
public enum PermType
{{
    ExplicitDeny = 0,
    ExplicitAllow = 1,
    InheritedDeny = 2,
    InheritedAllow = 3,
    Unspecified = 4
}}
\\"@


Add-Type -TypeDefinition @\\"
public enum AclInheritance
{{
    Explicit = 0,
    Inherited = 1,
    None = 2,
}}
\\"@


# Convert a GUID to a display name
function GuidToDisplayName($guid)
{{
    $rootdse = Get-ADRootDSE
    $domain = Get-ADDomain

    # Create a hashtable to store the GUID value of each schema class and
    # attribute
    $guidmap = @{{}}
    Get-ADObject -SearchBase ($rootdse.SchemaNamingContext) -LDAPFilter `
    \\"(schemaidguid=*)\\" -Properties lDAPDisplayName,schemaIDGUID | 
    % {{$guidmap[$_.lDAPDisplayName]=[System.GUID]$_.schemaIDGUID}}

    foreach ($tuple in $guidmap.GetEnumerator())
    {{
        if ($tuple.Value -eq $guid)
        {{
            return $tuple.Key
        }}
    }}

    return $guid
}}


# Convert an extended rights GUID to display name
function ExtendedRightToDisplayName($er)
{{
    $rootdse = Get-ADRootDSE
    $domain = Get-ADDomain

    # Create a hashtable to store the GUID value of each extended right in the
    # forest
    $extendedrightsmap = @{{}}
    Get-ADObject -SearchBase ($rootdse.ConfigurationNamingContext) -LDAPFilter `
    \\"(&(objectclass=controlAccessRight)(rightsguid=*))\\" -Properties displayName,rightsGuid | 
    % {{$extendedrightsmap[$_.displayName]=[System.GUID]$_.rightsGuid}}

    foreach ($tuple in $extendedrightsmap.GetEnumerator())
    {{
        if ($tuple.Value -eq $er)
        {{
            return $tuple.Key
        }}
    }}

    return $er
}}


# Check if an AD object is a group
function isGroup($groupName)
{{
    try
    {{
        $isGroup = get-adgroup $groupName
        return $true
    }}
    catch
    {{
        return $false
    }}
}}


# Check if an ACL applies to a user
function aclAppliesToUser($acl, $user)
{{
    $identityName = $acl.IdentityReference.ToString()

    # Break the identity name into domain and name
    $slashIndex = $identityName.IndexOf('\\')
    $identityDomain = $env:USERDOMAIN
    $identityShortName = $identityName

    if ($slashIndex -gt 0)
    {{
        $identityDomain = $identityName.Substring(0, $slashIndex)
        $identityShortName = $identityName.Substring($slashIndex + 1)
    }}

    # If the identity is for this domain and has the same username as the
    # account we are looking for, then this ACL applies
    if (($identityDomain.ToLower() -eq $env:USERDOMAIN.ToLower()) -and ($identityShortName.ToLower() -eq $user.ToLower()))
    {{
        return [AclInheritance]::Explicit
    }}

    # If this is a group, check if it is a member
    if (isGroup $identityShortName)
    {{
        # User identity should be a member of the group
        $members = Get-ADGroupMember $identityShortName -Recursive

        ForEach ($member in $members)
        {{
            if ($member.SamAccountName.ToLower() -eq $user.ToLower())
            {{
                return [AclInheritance]::Inherited
            }}
        }}
    }}

    # If account is 'everyone', then the right is inherited
    if ($identityShortName.ToLower() -eq 'everyone')
    {{
        return [AclInheritance]::Inherited
    }}

    return [AclInheritance]::None
}}


# Update if a right is enabled / disabled. Priority is as follows:
#  - Explicitly disabled
#  - Explicitly allowed
#  - Inherited disable
#  - Inherited allow
#  - Unspecified (which effectively is disabled)
function updateRights($currentPermType, $newPermission, $isInherited)
{{
    $newPermType = [PermType]::Unspecified

    # Convert inputs to PermType value
    if ($isInherited)
    {{
        if ($newPermission.Trim().ToLower() -eq \\"allow\\")
        {{
            $newPermType = [PermType]::InheritedAllow
        }}
        else
        {{
            $newPermType = [PermType]::InheritedDeny
        }}
    }}
    else
    {{
        if ($newPermission.Trim().ToLower() -eq \\"allow\\")
        {{
            $newPermType = [PermType]::ExplicitAllow
        }}
        else
        {{
            $newPermType = [PermType]::ExplicitDeny
        }}
    }}

    # Only update current perm if the new perm has higher priority
    if ($newPermType -lt $currentPermType)
    {{
        return $newPermType
    }}
    else
    {{
        return $currentPermType
    }}
}}


# Check if a user has the requested permissions on the specified OU
function userHasPermsOnOU($user, $perm, $ou)
{{
    $resRights = new-object PermType[] $perm.Rights.Count
    $acls = get-acl -Path \\"AD:\$ou\\" | select-object -ExpandProperty Access    

    foreach ($i in 0..($perm.Rights.Count - 1))
    {{
        $resRights[$i] = [PermType]::Unspecified
    }}

    ForEach ($acl in $acls)
    {{
        $inheritanceType = aclAppliesToUser $acl $user

        # Only look at this ACL if the user inherits any permissions from it
        if ($inheritanceType -ne [AclInheritance]::None)
        {{
            if ($acl.ActiveDirectoryRights.ToString().ToLower() -eq \\"extendedright\\")
            {{
                # Handle Extended Rights by first checking the inheritance levels
                if (($acl.InheritanceType.ToString().ToLower() -eq $perm.Inheritance.ToLower()) -or ($acl.InheritanceType.ToString().ToLower() -eq \\"all\\"))
                {{
                    $objectType = $(GuidToDisplayName($acl.InheritedObjectType)).Trim().ToLower()

                    # Check the object type matches or the rule applies to all objects
                    if ($objectType -eq \\"all\\" -or $objectType -eq $perm.Object.ToLower())
                    {{
                        # Check the permissions match
                        foreach ($permRightIndex in 0..($perm.Rights.Count - 1))
                        {{
                            $permRight = $perm.Rights[$permRightIndex].Trim().ToLower()
                            $sanitizedRight = $(ExtendedRightToDisplayName($acl.ObjectType)).Trim().ToLower()

                            if ($sanitizedRight -eq $permRight)
                            {{
                                # Update stored permissions
                                $resRights[$permRightIndex] = updateRights `
                                    -currentPermType $resRights[$permRightIndex] `
                                    -newPermission $acl.AccessControlType.ToString().Trim().ToLower() `
                                    -isInherited (($acl.IsInherited.ToString() -eq \\"True\\") -or ($inheritanceType -eq [AclInheritance]::Inherited))
                            }}
                        }}
                    }}
                }}
            }}
            else
            {{

                # Handle normal rights - start with checking if the perm
                # applies to the desired objects
                $objectType = $OBJECT_TYPE_MAP[$acl.ObjectType.ToString()].ToLower()

                if ($objectType -eq \\"all\\" -or $objectType -eq $perm.Object.ToLower())
                {{
                    # Check if the right is in the list of desired rights
                    $rights = $acl.ActiveDirectoryRights.ToString().Split(',')

                    foreach ($right in $rights)
                    {{
                        $sanitizedRight = $right.Trim().ToLower()

                        foreach ($permRightIndex in 0..($perm.Rights.Count - 1))
                        {{
                            $permRight = $perm.Rights[$permRightIndex].Trim().ToLower()

                            # Check if this is a generic right
                            $genericRightApplies = (
                                $sanitizedRight -eq 'genericall' -or
                                (
                                    $sanitizedRight -eq 'genericread' -and
                                    $permRight -Match 'read'
                                ) -or
                                (
                                    $sanitizedRight -eq 'genericwrite' -and
                                    $permRight -Match 'write'
                                )
                            )

                            if ($sanitizedRight -eq $permRight -or $genericRightApplies)
                            {{
                                # Update calculated permissions
                                $resRights[$permRightIndex] = updateRights `
                                    -currentPermType $resRights[$permRightIndex] `
                                    -newPermission $acl.AccessControlType.ToString().Trim().ToLower() `
                                    -isInherited (($acl.IsInherited.ToString() -eq \\"True\\") -or ($inheritanceType -eq [AclInheritance]::Inherited))
                            }}
                        }}
                    }}
                }}
            }}
        }}
    }}

    # Take resulting perms array and process if all perms are true
    $allPermsAllowed = $true

    foreach ($right in $resRights)
    {{
        if ($right -ne [PermType]::ExplicitAllow -and $right -ne [PermType]::InheritedAllow)
        {{
            $allPermsAllowed = $false
            break
        }}
    }}

    return $allPermsAllowed
}}


$USER = \\"{username}\\"
$OU=\\"{ou}\\"

$READ_WRITE_ALL_PROPERTIES_PERMS = New-Object PSObject -Property @{{
    Inheritance = \\"{inheritance}\\";
    Object = \\"{objects}\\";
    Rights = @({rights});
}}

userHasPermsOnOU $USER $READ_WRITE_ALL_PROPERTIES_PERMS $OU
"""

  rightsStr = ""

  for right in rights:
    if rightsStr and rightsStr != '':
      rightsStr += ', \\"' + right + '\\"'
    else:
      rightsStr +=  '\\"' + right + '\\"'

  return runPowershellCommand(cmd.format(
    username=username,
    ou=ou,
    inheritance=inheritance,
    objects=objects,
    rights=rightsStr))

In [ ]:
# Check an account has the requested permissions on an OU from a windows client
def accountHasPermsOnOUWindows(username, ou, inheritance, objects, rights):
  res = False
  error = 'Function did not finish'

  # Call powershell to get check account permissions
  stdout, stderr = powershellAccountHasPermsOnOUWindows(
    username, ou, inheritance, objects, rights)

  # Parse output
  if stderr and stderr != "":
    print("Failed to check if account '{0}' had permissions in ou '{1}': "\
          .format(username, ou) + stderr)
  else:
    error = ""

    if stdout.strip().lower() == 'true':
      res = True
    else:
      res = False

  return res


# Check that the input account has all permissions needed to deploy BDC from a
# windows client
def checkDeploymentPermsOfAccountWindows(username, ou):
  if not accountHasPermsOnOUWindows(
      username, ou, "all", "all", ["ReadProperty", "WriteProperty"]):
    print("User '{0}' needs to be able to read and write all properties on OU "
          "'{1}' and all its descendents. Please grant the necessary "
          "permissions to the deployment account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  elif not accountHasPermsOnOUWindows(
      username, ou, "all", "computer", ["CreateChild", "DeleteChild"]):
    print("User '{0}' needs to be able to create and delete child computers "
          "of OU '{1}'. Please grant the necessary permissions to the "
          "deployment account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  elif not accountHasPermsOnOUWindows(
      username, ou, "all", "user", ["CreateChild", "DeleteChild"]):
    print("User '{0}' needs to be able to create and delete child users of OU "
          "'{1}'. Please grant the necessary permissions to the deployment "
          "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  elif not accountHasPermsOnOUWindows(
      username, ou, "all", "group", ["CreateChild", "DeleteChild"]):
    print("User '{0}' needs to be able to create and delete child groups of "
          "OU '{1}'. Please grant the necessary permissions to the deployment "
          "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  elif not accountHasPermsOnOUWindows(
      username, ou, "descendents", "computer", ["reset password"]):
    print("User '{0}' needs to be able to reset passwords for descendant "
          "computers of OU '{1}'. Please grant the necessary permissions to "
          "the deployment account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  elif not accountHasPermsOnOUWindows(
      username, ou, "descendents", "user", ["reset password"]):
    print("User '{0}' needs to be able to reset passwords for descendant users"
          " of OU '{1}'. Please grant the necessary permissions to the "
          "deployment account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
          .format(username, ou))
  else:
    return True

  return False


# Check that the input account has all permissions needed to deploy BDC from a
# Linux client
def checkDeploymentPermsOfAccountLinux(username, password, ou):
  if not canCreateAccountInOU:
    display(Markdown("""Warning: because the notebook doesn't have permission to create test accounts, permissons of the AD BDC deployment account ({0}) cannot be checked. Please manually check it has the following permissions on the OU ({1}):  
- Read/Write all permissions for the OU and all descendents  
- Create/Delete child group objects in the OU and all descendents  
- Create/Delete child user objects in the OU and all descendents  
- Reset password for descendant user objects  
- Reset password for descendant computer objects""".format(username, ou)))

    return True

  testUser = "adNotebookUser"
  testGroup = "adNotebookGroup"
  testPwd = str(uuid.uuid4())

  # Login as the AD BDC deployment service account
  _, stderr = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && echo {1} | "
                             "kinit {0}".format(username, password))

  if stderr and stderr != '':
    print("Failed to login as user {0}, so could not check its permissions on "
          "OU {1}: {2}".format(username, ou, stderr))
    return False

  # Purge results of previous runs
  runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil user delete {0}"\
                 .format(testUser))
  runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil group delete "
                 "{0}".format(testGroup))

  # Test creating a user
  try:
    stdout, _ = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil "
                               "user create --distname CN={0},{1} --password "
                               "{2}".format(testUser, ou, testPwd))

    if stdout and stdout != "":
      print("Error: User {0} does not have permission to create a user in OU "
            "{1}. Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
            .format(username, ou))
      return False

    # Test creating a group
    stdout, _ = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil "
                               "group create --distname CN={0},{1}"\
                               .format(testGroup, ou))

    if stdout and stdout != "":
      print("Error: User {0} does not have permission to create a group in OU "
            "{1}. Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
            .format(username, ou))
      return False

    # Test setting an SPN
    stdout, _ = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil "
                               "spn add -s test/{0} -n {0}".format(testUser))

    if stdout and stdout != "":
      print("Error: User {0} does not have permission to set SPNs in OU {1}. "
            "Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
            .format(username, ou))
      return False

    # Test adding a user to a group
    stdout, _ = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil "
                               "group addmember -n {0} -g {1}"\
                               .format(testUser, testGroup))

    if stdout and stdout != "":
      print("Error: User {0} cannot add a user to a group in OU {1}. "
            "Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
            .format(username, ou))
      return False

    # Test enabling delegation and print warning (not error) if it fails
    stdout, _ = runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && adutil "
                               "delegation enable -n {0}".format(testUser))

    if not stdout or stdout.find("Successfully enabled delegation") != -1:
      print(
        "Warning: User '{0}' cannot enable delegation on an account in OU {1}. "
        "This is not mandatory, but some BDC functionality won't work after "
        "deployment such as linked servers between SQL instances"\
        .format(username, ou))

    # Test deleting a user
    stdout, _ = runBashCommand(
      "export KRB5CCNAME=/tmp/krb5ccache && adutil user delete -n {0}"\
      .format(testUser))

    if stdout and stdout != "":
      print(
        "Error: User {0} cannot delete a user from OU {1}. "
            "Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
        .format(username, ou))
      return False

    # Test deleting a group
    stdout, _ = runBashCommand(
      "export KRB5CCNAME=/tmp/krb5ccache && adutil group delete -n {0}"\
      .format(testGroup))

    if stdout and stdout != "":
      print(
        "Error: User {0} cannot delete a group from OU {1}. "
            "Please grant the necessary permissions to the deployment "
            "account: https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15#setting-permissions-the-bdc-ad-account"\
        .format(username, ou))
      return False

  finally:
    # Cleanup the accounts and credentials from our test
    runBashCommand(
      "export KRB5CCNAME=/tmp/krb5ccache && adutil user delete {0}"\
      .format(testUser))
    runBashCommand(
      "export KRB5CCNAME=/tmp/krb5ccache && adutil group delete {0}"\
      .format(testGroup))
    runBashCommand("export KRB5CCNAME=/tmp/krb5ccache && kdestroy")

  return True


# Check that the input account has all permissions needed to deploy BDC
def checkDeploymentPermsOfAccount(username, password, ou):
  # Permissions to check:
  # - Read / write all properties in OU
  # - Create / Delete computer objects in OU
  # - Create / Delete group objects in OU
  # - Create / Delete user objects in OU
  # - Reset passwords on descendant users
  # - Reset passwords on descendant computers
  if platform.system().lower() == 'windows':
    return checkDeploymentPermsOfAccountWindows(username, ou)
  else:
    return checkDeploymentPermsOfAccountLinux(username, password, ou)

In [ ]:
# Check that an account is enabled for windows client
def isUserEnabledWindows(username):
  isEnabled = False
  error = 'Function not finished'
  stdout, stderr = runPowershellCommand(
    "(get-aduser '{0}' -Properties enabled).enabled".format(username))

  # Parse output
  if stderr and stderr != '':
    error = stderr
  elif not stdout or stdout == '':
    # If nothing is returned the user probably does not have permissions.
    # Print a warning, but do not fail the test
    isEnabled = True
    error = "Warning: Do not have permissions to check if user '{0}' is " \
            "enabled. Please manually check this or try running Azure Data " \
            "Studio as administrator".format(username)
  else:
    # Check that the account disabled flag isn't set
    isEnabled = stdout.lower() == 'true'
    error = ''
    
  return isEnabled, error


# Check that a user is enabled for linux client
def isUserEnabledLinux(username):
  stdout, _ = runBashCommand("adutil account enabled {0}".format(username))
  sanitizedStdout = stdout.strip().lower()

  if sanitizedStdout == 'true':
    return True, ''
  elif sanitizedStdout == 'false':
    return False, ''
  else:
    return False, stdout


# Check that a user is enabled
def isUserEnabled(username):
  if platform.system().lower() == 'windows':
    return isUserEnabledWindows(username)
  else:
    return isUserEnabledLinux(username)


# Check that all accounts in the list are enabled
def checkAllAccountsEnabled(accountList):
  allEnabled = True

  for account in accountList:
    accountType, error = getAccountType(account)

    if error != '':
      allEnabled = False
      print("Failed to find type of account {0}: {1}".format(account, error))
    elif accountType == AccountType.User:
      enabled, error = isUserEnabled(account)

      if error and error != '':
        # Only fail if the account is not enabled. This is so we can print a
        # warning when we don't have the correct permissions to check the
        # user's status
        if enabled:
          print(error)
        else:
          allEnabled = False
          print("Failed to check if account '{0}' is enabled: {1}"\
                .format(account, error))
      elif not enabled:
        allEnabled = False
        print("Error: Account '{0}' is not enabled. Ensure it is enabled then "
              "run the notebook code cell again".format(account))
        break

  return allEnabled


# Validate a username and password combination on Windows client
def validateUsernamePasswordWindows(username, password):
  passwordCorrect = False
  error = 'Function not finished'
  stdout, stderr = runPowershellCommand(
    "New-Object System.DirectoryServices.DirectoryEntry($(\\\"LDAP://\\\" + "
    "([ADSI]'').distinguishedName), \\\"$env:USERDOMAIN\\{0}\\\", "
    "\\\"{1}\\\")".format(username, password))

  # Parse output
  if stderr and stderr != '':
    error = stderr
  elif not stdout or stdout == '':
    # If nothing is returned, report a bad password
    error = "Invalid username or password supplied for the BDC AD Domain " \
            "Service Account"
  else:
    # Success
    passwordCorrect = True
    error = ''

  return passwordCorrect, error


# Validate a username and password combination on Linux client
def validateUsernamePasswordLinux(username, password):
  stdout, stderr = runBashCommand(
    "export KRB5CCNAME=/tmp/krb5ccache && echo {0}| kinit {1} && kdestroy"\
    .format(password, username))

  if stderr == '':
    return True, ''
  else:
    return False, "Invalid username or password supplied for the BDC AD "\
                  "Domain Service Account"


# Validate a username and password combination
def validateUsernamePassword(username, password):
  res = False
  error = ''

  if platform.system().lower() == 'windows':
    res, error = validateUsernamePasswordWindows(username, password)
  else:
    res, error = validateUsernamePasswordLinux(username, password)

  if error and error != '':
    print(
      "Failed to check if password is correct for user {0}: {1}"\
      .format(username, error))
  elif not res:
    print("Incorrect password supplied for user {0}. Fix the BDC AD domain "
          "service account password input to this notebook then run the "
          "notebook again".format(username))

  return res

In [ ]:
# Test deployment accounts are valid
def deploymentAccountTest():
  res = True

  # Check that supplied groups are not domain-local
  res = res and checkNoLocalGroupsInList(clusterAdmins)
  res = res and checkNoLocalGroupsInList(clusterUsers)
  res = res and checkNoLocalGroupsInList(appOwners)

  # Check that groups and admin are enabled
  res = res and checkAllAccountsEnabled(clusterAdmins)
  res = res and checkAllAccountsEnabled(clusterUsers)
  res = res and checkAllAccountsEnabled(appOwners)
  res = res and checkAllAccountsEnabled([adDeploymentUsername])

  # Check admin password is correct
  res = res and validateUsernamePassword(adDeploymentUsername,
                                         adDeploymentPassword)

  # Check that deployment account has the correct perms
  res = res and checkDeploymentPermsOfAccount(
    adDeploymentUsername,
    adDeploymentPassword,
    ouDistinguishedName)

  return res


# Test deployment accounts
deploymentAccountChecksSucceeded = False
shouldRunTests = False

try:
  FIRST_CELL_RUN
  shouldRunTests = True
except NameError:
  print("Error: First code cell must be run before this cell")

if shouldRunTests:
  deploymentAccountChecksSucceeded = deploymentAccountTest()

if deploymentAccountChecksSucceeded:
  print("Success")

Summary
-------

In [ ]:
# Print summary of previous tests
def printStatus(prefix, succeeded):
  if succeeded:
    print(prefix + ": " + "\x1b[32mSUCCESS\x1b[0m")
  else:
    print(prefix + ": " + "\x1b[31mFAILURE\x1b[0m")


allTestsSucceeded = True

try:
  printStatus("Load Config Files", configTestsPassed)
  allTestSucceeded = allTestsSucceeded and configTestsPassed
except NameError:
  print("Load Config Files: NOT RUN")
  allTestsSucceeded = False

try:
  printStatus("DNS Tests", dnsTestsSucceeded)
  allTestsSucceeded = allTestsSucceeded and dnsTestsSucceeded
except NameError:
  print("DNS Tests: NOT RUN")
  allTestsSucceeded = False

try:
  printStatus("Existing Objects Tests", existingObjectsChecksSucceeded)
  allTestsSucceeded = allTestsSucceeded and existingObjectsChecksSucceeded
except NameError:
  print("Existing Objects Test: NOT RUN")
  allTestsSucceeded = False

try:
  printStatus("Deployment Account Tests", deploymentAccountChecksSucceeded)
  allTestsSucceeded = allTestsSucceeded and deploymentAccountChecksSucceeded
except NameError:
  print("Deployment Account Tests: NOT RUN")
  allTestsSucceeded = False

if not allTestsSucceeded:
  raise Exception('Not all tests passed')

In [ ]:
print("Notebook execution is complete.")